In [1]:
import pandas as pd
import altair as alt 
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from datetime import timedelta

alt.data_transformers.enable("vegafusion")
alt.themes.enable("fivethirtyeight")

ThemeRegistry.enable('fivethirtyeight')

In [2]:
import pandas as pd
import altair as alt 
import numpy as np
import datetime as dt
from datetime import timedelta

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [3]:
# Read in survey df 
def read_survey_df(): 
    df = pd.read_csv('/Users/ghazalin/NH_inspections_static/data/NH_SurveyDates_Sep2024.csv')
    df = df.rename(columns={'CMS Certification Number (CCN)':'CCN', 'Survey Date': 'survey_date','Type of Survey':'survey_type', 'Survey Cycle':'survey_cycle', 'Processing Date':'process_date'})
    print(df.head(10))
    return df 
survey_df = read_survey_df()

      CCN survey_date           survey_type  survey_cycle process_date
0  015009  2023-03-02      Health Complaint             2   2024-09-01
1  015009  2023-03-02       Health Standard             1   2024-09-01
2  015009  2023-02-15  Fire Safety Standard             1   2024-09-01
3  015009  2019-08-21  Fire Safety Standard             2   2024-09-01
4  015009  2019-08-21       Health Standard             2   2024-09-01
5  015009  2018-08-09  Fire Safety Standard             3   2024-09-01
6  015009  2018-08-01       Health Standard             3   2024-09-01
7  015010  2023-08-30     Infection Control             2   2024-09-01
8  015010  2022-04-13  Fire Safety Standard             1   2024-09-01
9  015010  2022-04-09       Health Standard             1   2024-09-01


In [4]:
def standard_surveys_only(df): 
    print(df['survey_type'].value_counts())
    df = df.loc[df['survey_type']=='Health Standard'].copy()
    print(df.head(10))
    print(df.shape[0])
    return df
survey_df = standard_surveys_only(survey_df)

survey_type
Health Complaint         54456
Health Standard          44249
Fire Safety Standard     44216
Infection Control        17878
Fire Safety Complaint      570
Name: count, dtype: int64
       CCN survey_date      survey_type  survey_cycle process_date
1   015009  2023-03-02  Health Standard             1   2024-09-01
4   015009  2019-08-21  Health Standard             2   2024-09-01
6   015009  2018-08-01  Health Standard             3   2024-09-01
9   015010  2022-04-09  Health Standard             1   2024-09-01
11  015010  2019-06-13  Health Standard             2   2024-09-01
13  015010  2018-06-07  Health Standard             3   2024-09-01
15  015012  2022-03-24  Health Standard             1   2024-09-01
17  015012  2019-06-06  Health Standard             2   2024-09-01
19  015012  2018-05-03  Health Standard             3   2024-09-01
22  015014  2023-09-13  Health Standard             1   2024-09-01
44249


In [17]:
survey_df['survey_date'] = pd.to_datetime(survey_df['survey_date'])
survey_df = survey_df.loc[survey_df['survey_date'].dt.year==2023]

In [18]:
# Read in provider df 
def read_provider_df(): 
    df = pd.read_csv('/Users/ghazalin/NH_inspections_static/data/NH_ProviderInfo_Sep2024.csv', dtype={'ZIP Code':str})
    df = df.rename(columns={'Date First Approved to Provide Medicare and Medicaid Services':'date_approved','CMS Certification Number (CCN)':'CCN','Provider Name':'provider_name','Provider Address':'provider_address','ZIP Code':'zip'})
    # print(df.columns.to_list())
    df = df[['CCN','date_approved','provider_name','State','zip']]
    df['zip'] = df['zip'].astype(str)

    return df

provider_df = read_provider_df()


In [31]:

survey_provider = pd.merge(survey_df, provider_df, how='left',on='CCN', indicator=True)
survey_provider = survey_provider.loc[survey_provider['State'].isin(['GU','PR'])==False]
# print('survey_provider_df rows: ' + str(survey_provider.shape[0]))
# print(survey_provider['_merge'].value_counts())


In [50]:
def survey_heatmap(): 
    survey_provider['survey_count'] = 1 
    heat_map = alt.Chart(survey_provider).mark_rect().encode(alt.X('yearmonth(survey_date)', title='Date'),
                                                             alt.Y('State:N'), 
                                                             color=alt.Color('sum(survey_count):Q', title='Survey Count').sort('ascending').scale(scheme='browns')
                                                            ).properties(background='white', title=alt.TitleParams(text='2023 Survey Counts by State', fontSize=15, dx=100))
    return heat_map
survey_heatmap().save('/Users/ghazalin/NH_inspections_static/images/survey_count_heatmap.svg')